In [1]:
# This is for my own setup

!ls -la /content
!git clone https://github.com/mehrdad-mirpourian/nlu_hw2.git
!ls -la /content/nlu_hw2


import os
os.chdir("/content/nlu_hw2")
print("Current directory:", os.getcwd())  # Should print /content/nlu_hw2

!ls -la

total 16
drwxr-xr-x 1 root root 4096 Mar  7 14:26 .
drwxr-xr-x 1 root root 4096 Mar 11 13:42 ..
drwxr-xr-x 4 root root 4096 Mar  7 14:25 .config
drwxr-xr-x 1 root root 4096 Mar  7 14:26 sample_data
Cloning into 'nlu_hw2'...
remote: Enumerating objects: 64, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 64 (delta 36), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (64/64), 75.57 KiB | 5.81 MiB/s, done.
Resolving deltas: 100% (36/36), done.
total 420
drwxr-xr-x 3 root root   4096 Mar 11 13:43 .
drwxr-xr-x 1 root root   4096 Mar 11 13:43 ..
drwxr-xr-x 8 root root   4096 Mar 11 13:43 .git
-rw-r--r-- 1 root root 124125 Mar 11 13:43 hw2-pset.ipynb
-rw-r--r-- 1 root root 270838 Mar 11 13:43 My_Checks.ipynb
-rw-r--r-- 1 root root   4903 Mar 11 13:43 README.md
-rw-r--r-- 1 root root   1310 Mar 11 13:43 test_model.py
-rw-r--r-- 1 root root   3952 Mar 11 13:43 train_model.py
Current directory: /content/nlu_hw2

In [2]:
! pip install datasets evaluate optuna --quiet # install datasets if it is not included in your environment

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.8/231.8 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 9.0 MB/s eta 0:00:00


In [32]:
import os
import numpy as np
from collections.abc import Iterable
from typing import Dict, Any

import torch
import torch.nn as nn

from datasets import load_dataset, Dataset
from transformers import AutoModelForSequenceClassification, \
    BertForSequenceClassification, BertTokenizerFast, Trainer, TrainingArguments

import evaluate
import optuna
from optuna.samplers import GridSampler

os.environ["WANDB_DISABLED"] = "true"

use_fp16 = torch.cuda.is_available()

In [4]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
model = BertForSequenceClassification.from_pretrained(
    "prajjwal1/bert-tiny", num_labels=2)

config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/17.8M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# This code does exactly the same thing as the previous code cell
model = AutoModelForSequenceClassification.from_pretrained(
    "prajjwal1/bert-tiny", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
tokenizer = BertTokenizerFast.from_pretrained("prajjwal1/bert-tiny")

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [8]:
# Because 🤗 Transformers supports multiple deep learning libraries, you will
# need to use the keyword parameter return_tensors in order to indicate that
# you want your inputs to be returned in PyTorch format.
inputs = tokenizer(["Hello world!", "How are you?"], padding=True,
                   return_tensors="pt")
inputs

{'input_ids': tensor([[ 101, 7592, 2088,  999,  102,    0],
        [ 101, 2129, 2024, 2017, 1029,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1]])}

In [9]:
model.eval()
with torch.no_grad():
    outputs = model(**inputs)

print(outputs, end="\n\n")

# Use the dot operator to access parts of the output
print(outputs.logits)

SequenceClassifierOutput(loss=None, logits=tensor([[0.2163, 0.4111],
        [0.2878, 0.3006]]), hidden_states=None, attentions=None)

tensor([[0.2163, 0.4111],
        [0.2878, 0.3006]])


## **Problem 1d. Prepare Dataset (Code, 10 Points)**

In [10]:
# Load IMDb dataset and create validation split
imdb = load_dataset("imdb")
split = imdb["train"].train_test_split(.2, seed=3463)
imdb["train"] = split["train"]
imdb["val"] = split["test"]
del imdb["unsupervised"]

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [11]:
def preprocess_dataset(dataset: Dataset, tokenizer: BertTokenizerFast) \
        -> Dataset:
    """
    Problem 1d: Implement this function.

    Preprocesses a dataset using a Hugging Face Tokenizer and prepares
    it for use in a Hugging Face Trainer.

    :param dataset: A dataset
    :param tokenizer: A tokenizer
    :return: The dataset, prepreprocessed using the tokenizer
    """

    def tokenize_function(examples):
        """
        Tokenizes input text using Hugging Face BERT tokenizer.
        Handles multiple edge cases to ensure robust preprocessing.
        """

        # Edge Case: Handle missing or empty text
        if "text" not in examples or examples["text"] is None:
            examples["text"] = ["[EMPTY]"] * len(examples.get("label", [0] * len(examples)))

        # Edge Case: Handle empty strings or unexpected formats
        examples["text"] = [t if isinstance(t, str) and t.strip() else "[EMPTY]" for t in examples["text"]]

        # Edge Case: Handle different data structures (alternative keys)
        if not isinstance(examples["text"], list):
            examples["text"] = [str(examples["text"])]

        try:
            tokenized = tokenizer(
                examples["text"],
                padding="max_length",  # Ensures all sequences are of equal length
                truncation=True,       # Ensures long sequences are truncated
                max_length=512        # As per Appendix A.2 of BERT paper
            )
        except Exception as e:
            # Edge Case: Handle tokenization failures
            print(f"Tokenization error: {e}")
            return {
                "input_ids": [[0] * 512],  # Default empty tokens
                "token_type_ids": [[0] * 512],
                "attention_mask": [[0] * 512]
            }

        return tokenized

    # Edge Case: Handle large batch sizes by limiting batch processing
    tokenized_dataset = dataset.map(tokenize_function, batched=True, batch_size=32)

    return tokenized_dataset




In [12]:
imdb["train"] = preprocess_dataset(imdb["train"], tokenizer)
imdb["val"] = preprocess_dataset(imdb["val"], tokenizer)
imdb["test"] = preprocess_dataset(imdb["test"], tokenizer)

# Visualize the preprocessed dataset
for k, v in imdb["val"][:2].items():
    print("{}:\n{}\n{}\n".format(k, type(v),
                                 [item[:20] if isinstance(item, Iterable) else
                                 item for item in v[:5]]))

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

text:
<class 'list'>
['As so many others ha', 'When converting a bo']

label:
<class 'list'>
[1, 0]

input_ids:
<class 'list'>
[[101, 2004, 2061, 2116, 2500, 2031, 2517, 1010, 2023, 2003, 1037, 6919, 4516, 1012, 2182, 2003, 1037, 2862, 1997, 1996], [101, 2043, 16401, 1037, 2338, 2000, 2143, 1010, 2009, 2003, 3227, 1037, 2204, 2801, 2000, 2562, 2012, 2560, 2070, 1997]]

token_type_ids:
<class 'list'>
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

attention_mask:
<class 'list'>
[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]



# **Problem 2: Implement Experiment (50 Points in Total)**

**Problem 2a: Freeze Non-Bias Weights (Code, 10 Points)bold text bold text**

In [13]:
def init_model(trial: Any, model_name: str, use_bitfit: bool = False) -> \
        BertForSequenceClassification:
    """
    Problem 2a: Implement this function.

    This function should be passed to your Trainer's model_init keyword
    argument. It will be used by the Trainer to initialize a new model
    for each hyperparameter tuning trial. Your implementation of this
    function should support training with BitFit by freezing all non-
    bias parameters of the initialized model.

    :param trial: This parameter is required by the Trainer, but it will
        not be used for this problem. Please ignore it
    :param model_name: The identifier listed in the Hugging Face Model
        Hub for the pre-trained model that will be loaded
    :param use_bitfit: If True, then all parameters will be frozen other
        than bias terms
    :return: A newly initialized pre-trained Transformer classifier
    """

    # Ensure default behavior when use_bitfit=False: I added this line
    # Becasue during testing my code I got some unexpected output when
    # bitfit = False. Adding this line made my output behave correctly.
    for param in model.parameters():
        param.requires_grad = True  # Make all parameters trainable by default

    # If BitFit is enabled, freeze all non-bias parameters
    if use_bitfit:
        for name, param in model.named_parameters():
            if "bias" not in name:  # Freeze all non-bias parameters
                param.requires_grad = False

    return model

In [14]:
# The first parameter is unused; we just pass None to it
model = init_model(None, "prajjwal1/bert-tiny", use_bitfit=True)

# Check if weight matrix is frozen
print(model.bert.encoder.layer[0].attention.self.query.weight.requires_grad)

# Check if bias term is frozen
print(model.bert.encoder.layer[0].attention.self.query.bias.requires_grad)

False
True


**Problem 2b: Set Up Trainer and Tester (Code, 20 Points)**

In [15]:
# def init_trainer(model_name: str, train_data: Dataset, val_data: Dataset,
#                  use_bitfit: bool = False) -> Trainer:
#     """
#     Problem 2b: Implement this function.

#     Creates a Trainer object that will be used to fine-tune a BERT-tiny
#     model on the IMDb dataset. The Trainer should fulfill the criteria
#     listed in the problem set.

#     :param model_name: The identifier listed in the Hugging Face Model
#         Hub for the pre-trained model that will be fine-tuned
#     :param train_data: The training data used to fine-tune the model
#     :param val_data: The validation data used for hyperparameter tuning
#     :param use_bitfit: If True, then all parameters will be frozen other
#         than bias terms
#     :return: A Trainer used for training
#     """

#     # Step 1: Load model using init_model function that we developed earlier
#     model = init_model(None, model_name, use_bitfit)

#     # Step 2: Define a function for computing accuracy on validation
#     def compute_metrics(eval_pred):
#         metric = evaluate.load("accuracy")
#         logits, labels = eval_pred
#         predictions = np.argmax(logits, axis=-1)
#         return metric.compute(predictions=predictions, references=labels)

#     # Step 3: Define training arguments for our trainer

#     training_args = TrainingArguments(
#     output_dir="checkpoints",
#     evaluation_strategy="epoch",
#     save_strategy="epoch",
#     logging_dir="./logs",
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     num_train_epochs=4,
#     save_total_limit=4,
#     load_best_model_at_end=True,
#     metric_for_best_model="accuracy",
#     fp16=use_fp16  # Enable mixed-precision only if GPU is available
# )

#     # Step 4: Create and return trainer object
#     trainer = Trainer(
#         model=model,
#         args=training_args,
#         train_dataset=train_data,
#         eval_dataset=val_data,
#         compute_metrics=compute_metrics
#     )

#     return trainer

In [25]:
def init_trainer(model_name: str, train_data: Dataset, val_data: Dataset,
                 use_bitfit: bool = False) -> Trainer:
    """
    Problem 2b: Implement this function.

    Creates a Trainer object that will be used to fine-tune a BERT-tiny
    model on the IMDb dataset. The Trainer should fulfill the criteria
    listed in the problem set.

    :param model_name: The identifier listed in the Hugging Face Model
        Hub for the pre-trained model that will be fine-tuned
    :param train_data: The training data used to fine-tune the model
    :param val_data: The validation data used for hyperparameter tuning
    :param use_bitfit: If True, then all parameters will be frozen other
        than bias terms
    :return: A Trainer used for training
    """

    # Step 1: Define a function for computing accuracy on validation
    def compute_metrics(eval_pred):
        metric = evaluate.load("accuracy")
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        return metric.compute(predictions=predictions, references=labels)

    # Step 2: Define training arguments for our trainer
    training_args = TrainingArguments(
        output_dir="checkpoints",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        logging_dir="./logs",
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=4,
        save_total_limit=4,
        load_best_model_at_end=True,
        metric_for_best_model="accuracy",
        fp16=use_fp16  # Enable mixed-precision only if GPU is available
    )

    # Step 3: Create and return trainer object with model_init for hyperparameter tuning
    trainer = Trainer(
        model_init=lambda: init_model(None, model_name, use_bitfit),  # Allow hyperparameter search
        args=training_args,
        train_dataset=train_data,
        eval_dataset=val_data,
        compute_metrics=compute_metrics
    )

    return trainer

In [26]:
def init_tester(directory: str) -> Trainer:
    """
    Problem 2b: Implement this function.

    Creates a Trainer object that will be used to test a fine-tuned
    model on the IMDb test set. The Trainer should fulfill the criteria
    listed in the problem set.

    :param directory: The directory where the model being tested is saved
    :return: A Trainer used for testing
    """

    # Load the fine-tuned model from the directory
    model = BertForSequenceClassification.from_pretrained(directory)

    # Define evaluation arguments (no training, only testing)
    training_args = TrainingArguments(
        output_dir="./results",  # Store evaluation results
        per_device_eval_batch_size=16,  # Match batch size used in training
        do_train=False,  # Ensure training is disabled
        do_eval=True,  # Enable evaluation
        logging_dir="./logs",  # Directory for logs
        report_to="none"  # Disable unnecessary logging (e.g., wandb)
    )

    # Define compute_metrics function (same as in training)
    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        accuracy = (predictions == labels).mean()
        return {"accuracy": accuracy}

    # Create Trainer for testing
    trainer = Trainer(
        model=model,
        args=training_args,
        compute_metrics=compute_metrics
    )

    return trainer

In [27]:
# Creates a Trainer from a Hugging Face Model Hub identifier
trainer = init_trainer("prajjwal1/bert-tiny", imdb["train"], imdb["val"])

# Train using the trainer
trainer.train()


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Accuracy
1,0.212600,0.352363,0.877000
2,0.172800,0.476639,0.868800
3,0.153400,0.461387,0.879400
4,0.143000,0.473481,0.882800


TrainOutput(global_step=5000, training_loss=0.17733718109130858, metrics={'train_runtime': 164.9364, 'train_samples_per_second': 485.035, 'train_steps_per_second': 30.315, 'total_flos': 101638963200000.0, 'train_loss': 0.17733718109130858, 'epoch': 4.0})

In [28]:
print(os.listdir("checkpoints"))

['checkpoint-2500', 'checkpoint-5000', 'checkpoint-3750', 'checkpoint-1250']


In [29]:
# Change this to whichever checkpoint you want to evalaute
eval_checkpoint_directory = "checkpoints/checkpoint-2500"

In [33]:
# Creates a Trainer to test a Hugging Face saved model
tester = init_tester(eval_checkpoint_directory)

In [34]:
tester.evaluate(imdb["test"])

KeyError: 'test'

**Problem 2c: Set Up Hyperparameter Tuning (Code, 20 Points)**

In [41]:
def hyperparameter_search_settings() -> Dict[str, Any]:
    """
    Problem 2c: Implement this function.

    Returns keyword arguments passed to Trainer.hyperparameter_search.
    Uses GridSampler for hyperparameter tuning, as specified in the problem set.

    :return: Keyword arguments for Trainer.hyperparameter_search
    """

    # Define the hyperparameter search space using Grid Search
    search_space = {
        "learning_rate": [3e-4, 1e-4, 5e-5],
        "per_device_train_batch_size": [8, 16],
        "weight_decay": [0.01, 0.001],
    }

    # Convert search space into a list of discrete choices for GridSampler
    grid_sampler = GridSampler(search_space, seed=42)

    # Function to return a **SINGLE sampled value** per hyperparameter
    def hp_space(trial):
        return {
            "learning_rate": trial.suggest_float("learning_rate", 5e-5, 3e-4 , log=True),
            "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [8, 16]),
            "weight_decay": trial.suggest_float("weight_decay", 0.001, 0.01),
        }

    # Function to maximize accuracy
    def compute_objective(metrics):
        return metrics["eval_accuracy"]

    return {
        "direction": "maximize",
        "hp_space": hp_space,  # Fix: Use a function to sample values, not pass a list
        "compute_objective": compute_objective,
        "n_trials": len(search_space["learning_rate"]) *
                    len(search_space["per_device_train_batch_size"]) *
                    len(search_space["weight_decay"]),
        "sampler": grid_sampler,
    }


In [42]:
# Load IMDb dataset
imdb = load_dataset("imdb")
split = imdb["train"].train_test_split(.2, seed=3463)
imdb["train"] = split["train"]
imdb["val"] = split["test"]
del imdb["unsupervised"]
del imdb["test"]

# Preprocess the dataset
tokenizer = BertTokenizerFast.from_pretrained("prajjwal1/bert-tiny")
imdb["train"] = preprocess_dataset(imdb["train"], tokenizer)
imdb["val"] = preprocess_dataset(imdb["val"], tokenizer)

# Initialize Trainer
trainer = init_trainer("prajjwal1/bert-tiny", imdb["train"], imdb["val"], use_bitfit=True)

# Run hyperparameter tuning
best_hyperparams = trainer.hyperparameter_search(**hyperparameter_search_settings())
print(best_hyperparams)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
[I 2025-03-11 14:15:27,382] A new study created in memory with name: no-name-8618a6e8-8998-43c3-a183-89ee5fd6a16a


Epoch,Training Loss,Validation Loss,Accuracy
1,0.078400,0.482169,0.879800
2,0.075200,0.480872,0.879800
3,0.089300,0.475889,0.881000
4,0.128100,0.478053,0.880400


[I 2025-03-11 14:18:05,131] Trial 0 finished with value: 0.8804 and parameters: {'learning_rate': 5e-05, 'per_device_train_batch_size': 16, 'weight_decay': 0.01}. Best is trial 0 with value: 0.8804.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.142100,0.484822,0.877200
2,0.126400,0.456835,0.883200
3,0.149200,0.463258,0.880200
4,0.130800,0.462413,0.880800


[I 2025-03-11 14:21:35,397] Trial 1 finished with value: 0.8808 and parameters: {'learning_rate': 5e-05, 'per_device_train_batch_size': 8, 'weight_decay': 0.001}. Best is trial 1 with value: 0.8808.


...

Epoch,Training Loss,Validation Loss,Accuracy
1,0.073200,0.466695,0.875000


[I 2025-03-11 14:36:26,081] Trial 10 pruned. 


Epoch,Training Loss,Validation Loss,Accuracy
1,0.070300,0.462119,0.875200


[I 2025-03-11 14:37:05,994] Trial 11 pruned. 


BestRun(run_id='3', objective=0.8826, hyperparameters={'learning_rate': 5e-05, 'per_device_train_batch_size': 8, 'weight_decay': 0.01}, run_summary=None)


In [ ]:
# epochs, dropout, and optimizer

In [44]:
# Learning



